In [ ]:
import os

In [ ]:
os.environ

In [ ]:
migs= 'MIG-c3ce33ce-ced8-5961-bb87-2b40eb100277,MIG-280489c4-1d98-5b07-b4f6-2fc85fc874fa,MIG-c432df19-0894-5232-ac1c-9a3440fc267e,MIG-e8f61a95-352a-56cc-b95d-0c35fc14e8bf'

In [ ]:
env = dict(os.environ)

In [ ]:
env['CUDA_VISIBLE_DEVICES']=migs

In [ ]:
os.environ = env

In [ ]:
os.environ

In [1]:
import os
import ray

# Set CUDA_VISIBLE_DEVICES to control which GPUs are visible to Ray
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3,4,5,6"

# Initialize Ray with the appropriate configuration
ray.init(
    num_cpus=4,
    num_gpus=7,
    resources={
        "gpu": 1
    },
    _system_config={
        "num_heartbeats_timeout": 120  # Increase heartbeat timeout to avoid spurious errors
    }
)


2023-02-24 18:34:09,244	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8266 


Python version:,3.9.16
Ray version:,2.2.0
Dashboard:,http://127.0.0.1:8266


In [2]:
resources = ray.cluster_resources()
print(resources)


{'GPU': 7.0, 'accelerator_type:A100': 1.0, 'node:158.125.168.72': 1.0, 'gpu': 1.0, 'memory': 153521763328.0, 'object_store_memory': 70080755712.0, 'CPU': 64.0}


In [3]:
import numpy as np
import gym
from gym_CTgraph import CTgraph_env
from gym_CTgraph.CTgraph_plot import CTgraph_plot
from gym_CTgraph.CTgraph_conf import CTgraph_conf
from gym_CTgraph.CTgraph_images import CTgraph_images
import json
import argparse
#import json
import random
import matplotlib.pyplot as plt
import timeit

from gym.spaces import Box, Discrete, Dict
from ray.tune.registry import register_env
from ray import tune

from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import argparse
import gym
from gym.spaces import Discrete, Box
import numpy as np
#import os
import random

import ray
from ray import tune
from ray.rllib.agents import ppo
from ray.rllib.env.env_context import EnvContext
from ray.rllib.models import ModelCatalog
from ray.rllib.models.tf.tf_modelv2 import TFModelV2
from ray.rllib.models.tf.fcnet import FullyConnectedNetwork
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.models.torch.fcnet import FullyConnectedNetwork as TorchFC
from ray.rllib.utils.framework import try_import_tf, try_import_torch
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.tune.logger import pretty_print

Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


/home/lunet/cocp5/.conda/envs/rllib20a/lib/python3.9/site-packages/tensorflow_probability/python/__init__.py:61: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if (distutils.version.LooseVersion(tf.__version__) <


In [4]:
'''import ray
ray.init()
resources = ray.cluster_resources()
print(resources)'''


'import ray\nray.init()\nresources = ray.cluster_resources()\nprint(resources)'

In [5]:
from ray import tune

In [6]:
def env_creator(env_config={}):
    # fetch the parameters from the json file
    configuration = CTgraph_conf("/home/lunet/cocp5/Documents/rllib20/graph09.json")
    conf_data = configuration.getParameters()
    # print configration data
    print(json.dumps(conf_data, indent=3))
    env = gym.make('CTgraph-v0')
    # instantiate the maze
    #start = timeit.timeit()
    #env = gym.make('CTgraph-v0')
    #end = timeit.timeit()
    #print(end - start)

    imageDataset = CTgraph_images(conf_data)
    observation = env.init(conf_data, imageDataset)

    # get a random path from the maze
    #high_reward_path = np.array(np.random.randint(1, conf_data['graph_shape']['branching_factor'], size=conf_data['graph_shape']['depth']))
    # use this random path to set the path to the high reward. Note that the maze would have already a high_reward_path from the initialisation
    #env.set_high_reward_path(high_reward_path)
    #seed_everything(env)  
    
    return env

In [7]:
register_env("CT-Graph_d2_b2_wp05_crv0_MDP", env_creator)

In [ ]:
tune.run("PPO",
             # algorithm specific configuration
             config={"env": "CT-Graph_d2_b2_wp05_crv0_MDP",  #
                     "framework": "torch",
                     "seed":958,
                     #"num_gpus": 7,
                     "num_rollout_workers": 1,
                     "num_envs_per_worker": 1,
                     "num_gpus_per_worker": 0,
                     "train_batch_size": 512,
                     "use_critic":True,
                     "use_gae": True,
                     "lambda": 0.99,
                     "sgd_minibatch_size": 64,
                     "num_sgd_iter": 8,
                     "shuffle_sequences": False,
                     "entropy_coeff": 0.00015,
                     "clip_param": 0.1,
                     "grad_clip": 5,
                     "lr": 0.0015,
                     #"num_cpus_per_worker": 1,
                     #"model": {"custom_model": "pa_model", },
                     "evaluation_interval": 1,
                     "evaluation_num_episodes": 2
                     },
             local_dir="CT-Graph_d2_b2_wp05_crv0_MDP04",  # directory to save results
             checkpoint_freq=2,  # frequency between checkpoints
             keep_checkpoints_num=6,
             stop={"timesteps_total": 1024000}
             )


2023-02-24 18:34:15,720	WARNING deprecation.py:47 -- DeprecationWarning: `AlgorithmConfig.evaluation(evaluation_num_episodes=..)` has been deprecated. Use `AlgorithmConfig.evaluation(evaluation_duration=.., evaluation_duration_unit='episodes')` instead. This will raise an error in the future!
(pid=2262405) /home/lunet/cocp5/.conda/envs/rllib20a/lib/python3.9/site-packages/tensorflow_probability/python/__init__.py:61: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(pid=2262405)   if (distutils.version.LooseVersion(tf.__version__) <
(pid=2262405) WARNING:tensorflow:From /home/lunet/cocp5/.conda/envs/rllib20a/lib/python3.9/site-packages/tensorflow/python/util/deprecation.py:561: calling function (from tensorflow.python.eager.def_function) with experimental_relax_shapes is deprecated and will be removed in a future version.
(pid=2262405) Instructions for updating:
(pid=2262405) experimental_relax_shapes is deprecated, use reduce_retracing inste

(RolloutWorker pid=2262938) ---------------------------------------------------
(RolloutWorker pid=2262938)              The CT-graph environments             
(RolloutWorker pid=2262938) ---------------------------------------------------
(RolloutWorker pid=2262938) Reading configuration parameters from  /home/lunet/cocp5/Documents/rllib20/graph09.json
(RolloutWorker pid=2262938) {
(RolloutWorker pid=2262938)    "general_seed": 3,
(RolloutWorker pid=2262938)    "graph_shape": {
(RolloutWorker pid=2262938)       "depth": 2,
(RolloutWorker pid=2262938)       "branching_factor": 2,
(RolloutWorker pid=2262938)       "wait_prob": 0.5
(RolloutWorker pid=2262938)    },
(RolloutWorker pid=2262938)    "reward": {
(RolloutWorker pid=2262938)       "high_reward_value": 1.0,
(RolloutWorker pid=2262938)       "crash_reward_value": 0.0,
(RolloutWorker pid=2262938)       "stochastic_sampling": false,
(RolloutWorker pid=2262938)       "reward_std": 0.1,
(RolloutWorker pid=2262938)       "min_static_r

(PPO pid=2262405) /home/lunet/cocp5/.conda/envs/rllib20a/lib/python3.9/site-packages/gym/core.py:172: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed) instead.
(PPO pid=2262405)   deprecation(
(PPO pid=2262405) 2023-02-24 18:34:32,630	INFO trainable.py:172 -- Trainable.setup took 10.812 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
(PPO pid=2262405) 2023-02-24 18:34:32,630	WARNING util.py:66 -- Install gputil for GPU system monitoring.


Trial name,agent_timesteps_total,counters,custom_metrics,date,done,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,episodes_total,evaluation,experiment_id,hostname,info,iterations_since_restore,node_ip,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_trained,num_env_steps_trained_this_iter,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,pid,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,sampler_results,time_since_restore,time_this_iter_s,time_total_s,timers,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
PPO_CT-Graph_d2_b2_wp05_crv0_MDP_d76fc_00000,317952,"{'num_env_steps_sampled': 317952, 'num_env_steps_trained': 317952, 'num_agent_steps_sampled': 317952, 'num_agent_steps_trained': 317952}",{},2023-02-24_18-45-08,False,2.9883,{},0,0,0,171,39925,"{'episode_reward_max': 0.0, 'episode_reward_min': 0.0, 'episode_reward_mean': 0.0, 'episode_len_mean': 2.5, 'episode_media': {}, 'episodes_this_iter': 2, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [0.0, 0.0], 'episode_lengths': [2, 3]}, 'sampler_perf': {'mean_raw_obs_processing_ms': 0.28263861512600524, 'mean_inference_ms': 0.7838638088377607, 'mean_action_processing_ms': 0.040064897927297866, 'mean_env_wait_ms': 0.29244994897193427, 'mean_env_render_ms': 0.0}, 'num_faulty_episodes': 0, 'num_agent_steps_sampled_this_iter': 5, 'num_env_steps_sampled_this_iter': 5, 'timesteps_this_iter': 5, 'num_healthy_workers': 0, 'num_in_flight_async_reqs': 0, 'num_remote_worker_restarts': 0}",e6d54482566e4f5d8b9b6ebade7556f7,lnx-grid-19.lunet.lboro.ac.uk,"{'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'grad_gnorm': 1.1333811011863872, 'cur_kl_coeff': 2.4423202414465676e-131, 'cur_lr': 0.0014999999999999998, 'total_loss': 0.0017086309962905943, 'policy_loss': 0.0017518851382192224, 'vf_loss': 4.123641326847505e-06, 'vf_explained_var': -0.9904828853905201, 'kl': 0.0018772643442065405, 'entropy': 0.31589604541659355, 'entropy_coeff': 0.00015}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': 64.0, 'num_grad_updates_lifetime': 39712.5, 'diff_num_grad_updates_vs_sampler_policy': 31.5}}, 'num_env_steps_sampled': 317952, 'num_env_steps_trained': 317952, 'num_agent_steps_sampled': 317952, 'num_agent_steps_trained': 317952}",621,158.125.168.72,317952,317952,317952,512,317952,512,0,1,0,0,512,"{'cpu_util_percent': 4.9, 'ram_util_percent': 7.2}",2262405,{},{},{},"{'mean_raw_obs_processing_ms': 0.3209926519873138, 'mean_inference_ms': 0.7572539836433644, 'mean_action_processing_ms': 0.03783545223367014, 'mean_env_wait_ms': 0.27734407949219225, 'mean_env_render_ms': 0.0}","{'episode_reward_max': 0.0, 'episode_reward_min': 0.0, 'episode_reward_mean': 0.0, 'episode_len_mean': 2.9883040935672516, 'episode_media': {}, 'episodes_this_iter': 171, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [ ]:
>>> from ray.rllib.algorithms.ddppo import DDPPOConfig
>>> config = DDPPOConfig().training(lr=0.003, keep_local_weights_in_sync=True)
>>> config = config.resources(num_gpus=1)
>>> config = config.rollouts(num_rollout_workers=10)
>>> print(config.to_dict())   
>>> # Build a Algorithm object from the config and run 1 training iteration.
>>> algo = config.build(env="CartPole-v1")  
>>> algo.train()  